In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seawater
import xarray as xr
import scipy.signal as sp
import pandas as pd
import math
import sys
import time


np.set_printoptions(threshold=50)

%matplotlib inline

## Integrate PSSC in ONAV
This notebook contains the update as we move forward integrating Potential Sub-Surface channel variable in Navigator following option 3 (A fixed frequency cutoff value) 
### Importing the sample profile

In [ ]:
profile = pd.read_csv("sample-pssc-profiles/ts_giops_day_2021-01-22T17_20_15.578175_POSITIVE.csv", header=2)
profile.columns

In [ ]:
depth = profile['Depth (m)']
Profile_sspeed = profile['Sound Speed']
temperature = profile ['Temperature']
salinity = profile ['Salinity']
latitude = profile['Latitude']
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.plot(profile['Sound Speed'], profile['Depth (m)'])
ax.grid(True)
ax.set_ylim([1000,0])
ax.set_ylabel('Depth [m]')
ax.set_xlabel('Speed of sound [m/s]')

### Bringing in functions from Navigator
These functions below are from Ocean-Data-Map-Project/data/calculated_parser/functions.py

https://github.com/DFO-Ocean-Navigator/Ocean-Data-Map-Project/blob/master/data/calculated_parser/functions.py

In [ ]:
def __calc_pressure(depth, latitude):
    pressure = []
    try:
        pressure = [seawater.pres(d, latitude) for d in depth]
    except TypeError:
        pressure = seawater.pres(depth, latitude)

    return np.array(pressure)

def __validate_depth_lat_temp_sal(depth, latitude, temperature, salinity):

    if type(depth) is not np.ndarray:
        depth = np.array(depth)

    if type(latitude) is not np.ndarray:
        latitude = np.array(latitude)

    if type(temperature) is not np.ndarray:
        temperature = np.array(temperature)

    if type(salinity) is not np.ndarray:
        salinity = np.array(salinity)

    return depth, latitude, np.squeeze(temperature), np.squeeze(salinity)

def sspeed(depth, latitude, temperature, salinity):
    """
    Calculates the speed of sound.
    Required Arguments:
    * depth: The depth(s) in meters
    * latitude: The latitude(s) in degrees North
    * temperature: The temperatures(s) in Celsius
    * salinity: The salinity (unitless)
    """

    depth, latitude, temperature, salinity = __validate_depth_lat_temp_sal(
        depth, latitude, temperature, salinity)

    press = __calc_pressure(depth, latitude)

    if salinity.shape != press.shape:
        # pad array shape to match otherwise seawater freaks out
        press = press[..., np.newaxis]

    speed = seawater.svel(salinity, temperature, press)
    return np.squeeze(speed)

In [ ]:
calculated_sspeed = sspeed(depth, latitude, temperature, salinity)
#calc_speed.shape

### The functions to detect sub-surface channel (Single point calculation)

In [ ]:
def calculate_del_C(depth,latitude,temperature, salinity, freq_cutoff):
    """
     Calculate ΔC from a given sound profile and freq cutoff
     Required Arguments:
        * depth: The depth(s) in meters
        * latitude: The latitude(s) in degrees North
        * temperature: The temperatures(s) in Celsius
        * salinity: The salinity (unitless)
        * freq_cutoff: Desired frequency cutoff in Hz
     Returns the value of ΔC, which will later be used inside the 
    """
    depth, latitude, temperature, salinity = __validate_depth_lat_temp_sal(depth, latitude, temperature, salinity)
    
    soundspeed = sspeed(depth, latitude, temperature, salinity);
    print(soundspeed.shape)
    # Getting Cmin from the sound speed profile
    first_local_minimum = sp.find_peaks(-soundspeed)[0][0]
    Cmin = soundspeed[first_local_minimum]
    #Calculating del_Z
    local_maximum = sp.find_peaks(soundspeed)[0][0]
    channel_start_depth = depth[local_maximum]
    channel_end_depth = np.interp(soundspeed[local_maximum], soundspeed, depth) 
    del_Z = channel_end_depth - channel_start_depth
    # print(channel_start_depth)
    # Final calculation of delC
    numerator = freq_cutoff * del_Z
    denominator = 0.2652 * Cmin
    final_denom = numerator/denominator
    final_denom =np.power(final_denom,2)
    delC = float(Cmin/final_denom)
    return delC

#calculate_del_C(depth,latitude, temperature, salinity, 2755.03);

In [ ]:
"""
     Detect if there is sub-surface channel. 
     Required Arguments:
        * depth: Depth in meters
        * sspeed: Sound speed in m/s
     Returns 1 if the profile has a sub-surface channel, 0 if the profile does not have a sub-surface channel
"""
def detect_potential_sub_surface_channel_v3(depth, latitude,temperature, salinity, freq_cutoff = 2755.03 )-> bool:
    has_PSSC = 0
    del_C = calculate_del_C(depth,latitude,temperature, salinity, freq_cutoff)
    # Trimming the profile considering the depth above 1000m
    depth = depth[depth<1000]
    sspeed = sspeed[0:(len(depth))]
    # detecting the local minima and local maxima for the sound speed profile
    local_minima = sp.find_peaks(-sspeed)[0] # get the index array of local minima
    local_maxima = sp.find_peaks(sspeed)[0] # get the index array of local maxima
    if len(local_minima)>=2: #if there are 2 or more minima
        p1 = 0 # surface
        p2 = local_minima[0] #first minimum
        if len(local_maxima)>=2: # if there are more than one maxima
            p1 = local_maxima[0] #first maximum
            p3 = local_maxima[1] #second maximum
        else: #only one local maximum
            p3 =  local_maxima[0] 
            if p3 < p2: # the only one maxima is higher in the water column than the minima
                has_PSSC=0
        # print("p1 p2 p3 : " +str(p1)+" "+str(p2)+" "+str(p3))
        p1_sound_speed = sound_speed[p1]
        p2_sound_speed = sound_speed[p2]
        p3_sound_speed = sound_speed[p3]
    
        c1 = abs(p1_sound_speed-p2_sound_speed) 
        c2 = abs(p3_sound_speed-p2_sound_speed)
        #print("c1 = "+ str(c1) +"m/s")
        #print("c2 = "+ str(c2) +"m/s")
    
        if c1> del_C and c2> del_C: # Changing this comparison to check with the calculated ΔC
            has_PSSC =1
        else:
            has_PSSC =0
    else:
        has_PSSC =0 
    #print(local_minima)
    #print(local_maxima)
    return has_PSSC

### Transforming the function for inputs from NetCDF file

In [ ]:
d = xr.open_dataset('data/2019100100_000_3D_ps5km60N.nc')
d

In [ ]:
depth = d.depth.values
temp = d.votemper.values
sal = d.vosaline.values
lat = d.latitude.values

In [ ]:
depth2 = depth[depth<1000]
depth2.shape
temp = temp-273.15


In [ ]:
temp2 = temp[:,0:len(depth2),:,:]
print(temp2.shape)
sal2 =  sal[:,0:len(depth2),:,:]
print(sal2.shape)

In [ ]:
#sound_speed = np.ma.array(sspeed(depth2, lat, temp2, sal2), fill_value=np.nan)
sound_speed = sspeed(depth2, lat, temp2, sal2)
#sound_speed

In [13]:
sound_speed.shape

(46, 1610, 1770)

In [14]:
start = time.time()
sound_speed.shape
minima = np.apply_along_axis(sp.find_peaks,0,-sound_speed)
end = time.time()
print(end-start)
minima.shape

41.3229660987854


(2, 1610, 1770)

__need to check it for an expected size of soundspeed__ 

In [15]:
sound_speed_sliced = sound_speed[:,200:286,-126:]
sound_speed_sliced.shape

(46, 86, 126)

In [16]:
start = time.time()
minima_alt = np.apply_along_axis(sp.find_peaks,0,-sound_speed_sliced)
end = time.time()
print(end-start)
minima_alt = minima_alt[0]
minima_alt


0.18620777130126953


array([[array([33]), array([33]), array([32]), ...,
        array([], dtype=int64), array([], dtype=int64), array([10, 29])],
       [array([33]), array([33]), array([33]), ...,
        array([], dtype=int64), array([29]), array([28])],
       [array([33]), array([33]), array([33]), ..., array([28]),
        array([27]), array([26])],
       ...,
       [array([], dtype=int64), array([], dtype=int64),
        array([], dtype=int64), ..., array([], dtype=int64), array([15]),
        array([], dtype=int64)],
       [array([], dtype=int64), array([], dtype=int64),
        array([], dtype=int64), ..., array([], dtype=int64),
        array([], dtype=int64), array([], dtype=int64)],
       [array([], dtype=int64), array([], dtype=int64),
        array([], dtype=int64), ..., array([], dtype=int64),
        array([], dtype=int64), array([], dtype=int64)]], dtype=object)

In [17]:
first_minimum = np.empty_like(minima_alt, dtype='int64')
first_minimum = minima_alt
start = time.time()
it = np.nditer(minima_alt,flags=['refs_ok','multi_index'])
for x in it:
    minima_size = x.tolist().size
    first_minimum[it.multi_index]= x.tolist()[0] if minima_size>0 else -1
end = time.time()
print(end-start)
first_minimum

0.019616127014160156


array([[33, 33, 32, ..., -1, -1, 10],
       [33, 33, 33, ..., -1, 29, 28],
       [33, 33, 33, ..., 28, 27, 26],
       ...,
       [-1, -1, -1, ..., -1, 15, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=object)

In [18]:
maxima_alt =  np.apply_along_axis(sp.find_peaks,0,sound_speed_sliced)
maxima_alt.shape

(2, 86, 126)

In [19]:
minima[0,:,:].shape

(1610, 1770)

In [20]:
minima2 = sp.argrelmin(sound_speed_sliced, axis= 0)
#minima2 = np.apply_along_axis(sp.argrelmin,1,-sound_speed)
minima2

(array([ 9,  9, 10, ..., 44, 44, 44]),
 array([ 0,  0,  0, ..., 47, 48, 48]),
 array([109, 112, 125, ...,  68,  65,  66]))

In [21]:
minima2[0].shape

(4761,)

In [22]:
#def calculate_del_C(depth, soundspeed, freq_cutoff):
def calculate_del_C(depth,soundspeed,minima,maxima, freq_cutoff):
    """
     Calculate ΔC from a given sound profile and freq cutoff
     Required Arguments:
        * depth: The depth(s) in meters
        * soundspeed: Speed of sound in m/s
        * minima: Minima ndarray of Speed of sound, which contains the index where the minima occurs
        * maxima: Maxima ndarray of Speed of sound,  which contains the index where the maxima occurs
        * freq_cutoff: Desired frequency cutoff in Hz
     Returns the value of ΔC, which will later be used inside the PSSC detection method
    """
    # Getting Cmin from the sound speed profile
    print(soundspeed.shape)
    #print(maxima.shape)
    first_minimum = np.empty_like(minima_alt, dtype='int64')
    it = np.nditer(minima,flags=['refs_ok','multi_index'])
    # need to look at alternative for this
    for x in it:
        minima_size = x.tolist().size
        first_minimum[it.multi_index]= x.tolist()[0] if minima_size>0 else -1
    print(first_minimum.shape)
    Cmin = soundspeed[first_minimum]
    #print(Cmin)
    ################
    Cmin = np.take_along_axis(soundspeed,first_minimum,axis=0)
    print(Cmin.shape)
    #Calculating del_Z
    local_maximum = sp.find_peaks(soundspeed)[0][0]
    channel_start_depth = depth[local_maximum]
    channel_end_depth = np.interp(soundspeed[local_maximum], soundspeed, depth) 
    del_Z = channel_end_depth - channel_start_depth
    # print(channel_start_depth)
    # Final calculation of delC
    numerator = freq_cutoff * del_Z
    denominator = 0.2652 * Cmin
    final_denom = numerator/denominator
    final_denom =np.power(final_denom,2)
    delC = float(Cmin/final_denom)
    return delC

#def potentialsubsurfacechannel(depth, latitude,temperature, salinity, freq_cutoff = 2755.03 )-> bool:
def potentialsubsurfacechannel(depth, sound_speed, freq_cutoff = 2755.03 )-> bool:   
    """
     Detect if there is sub-surface channel. 
     Required Arguments:
        * depth: Depth in meters
        * latitude: Latitude in degrees North
        * temperature: Temperatures in Celsius
        * salinity: Salinity
     Returns 1 if the profile has a sub-surface channel, 0 if the profile does not have a sub-surface channel
    """
    #print(depth)
    has_PSSC = 0
    # Trimming the profile considering the depth above 1000m
    #depth = depth[depth<1000]
    depth.shape[0]
    #temp = temperature[:,0:len(depth),:,:]
    #sal =  salinity[:,0:len(depth),:,:]
    #sound_speed = np.ma.array(sspeed(depth, latitude, temp, sal), fill_value=np.nan)
    minima = np.apply_along_axis(sp.find_peaks,0,-sound_speed)[0] # selecting the first item of the tuple
    maxima = np.apply_along_axis(sp.find_peaks,0, sound_speed)[0]
    del_C = calculate_del_C(depth,sound_speed,minima,maxima, freq_cutoff)
    
    
    #########
    
    sound_speed = sound_speed[0:(len(depth))]
    
    
    # detecting the local minima and local maxima for the sound speed profile
    local_minima = sp.find_peaks(-sound_speed)[0] # get the index array of local minima
    local_maxima = sp.find_peaks(sound_speed)[0] # get the index array of local maxima
    
    
    
    if len(local_minima)>=2: #if there are 2 or more minima
        p1 = 0 # surface
        p2 = local_minima[0] #first minimum
        if len(local_maxima)>=2: # if there are more than one maxima
            p1 = local_maxima[0] #first maximum
            p3 = local_maxima[1] #second maximum
        else: #only one local maximum
            p3 =  local_maxima[0] 
            if p3 < p2: # the only one maxima is higher in the water column than the minima
                has_PSSC=0
        # print("p1 p2 p3 : " +str(p1)+" "+str(p2)+" "+str(p3))
        p1_sound_speed = sound_speed[p1]
        p2_sound_speed = sound_speed[p2]
        p3_sound_speed = sound_speed[p3]
    
        c1 = abs(p1_sound_speed-p2_sound_speed) 
        c2 = abs(p3_sound_speed-p2_sound_speed)
        #print("c1 = "+ str(c1) +"m/s")
        #print("c2 = "+ str(c2) +"m/s")
    
        if c1> del_C and c2> del_C: # Changing this comparison to check with the calculated ΔC
            has_PSSC =1
        else:
            has_PSSC =0
    else:
        has_PSSC =0 
    #print(local_minima)
    #print(local_maxima)
    return has_PSSC


In [23]:
potentialsubsurfacechannel(depth, sound_speed_sliced, 2755.03)

(46, 86, 126)
(86, 126)


ValueError: `indices` and `arr` must have the same number of dimensions